# Solar Dynamics Observatory

Single timeseries of EUV data

### CASE 1: one chunk per file, timeseries sorted into filter bands. Doesn't need FITS.

In [ ]:
import intake
import hvplot.xarray

In [ ]:
cat = intake.open_catalog("https://github.com/fsspec/kerchunk/raw/main/"
                          "examples/intake_catalog.yml")

In [ ]:
ds = cat.SDO.to_dask()
ds

In [ ]:
ds.nbytes / 2**30  # GB

### What just happened?

Let's look into the detail a little.

First, where are the files we want:

In [ ]:
import fsspec
gcs = fsspec.filesystem("gcs")
gcs.ls("pangeo-data/SDO_AIA_Images/")

In [ ]:
gcs.ls("pangeo-data/SDO_AIA_Images/094")[:5]

Now we create a "reference" filesystem:

In [ ]:
fs = fsspec.filesystem(
    "reference",
    fo="gcs://mdtemp/SDO_no_coords.json",
    remote_options={"token": "anon"},
    remote_protocol="gcs",
    target_options={"token": "anon"}
)

In [ ]:
fs.ls("", False)

In [ ]:
fs.ls("094", False)[:5]

In [ ]:
print(fs.cat("094/.zarray").decode())

In [ ]:
fs.references["094/0.0.0"]

### Viz

So let's see it in action!

In [ ]:
import numpy as np
ds = ds.assign_coords(x=np.arange(4096), y=np.arange(4096))

In [ ]:
import numpy as np
ds["094"].hvplot.image(
    'x', 'y', colormap="fire", xaxis=False, yaxis=False, 
    height=500, width=500, colorbar=False, clim=(0, 50)
)